In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
url = 'https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?ref_=bo_lnav_hm_shrt'

response = requests.get(url)
page = response.text

soup = BeautifulSoup(page,"lxml")

In [4]:
response.status_code # build this in my code to ensure I am not getting errors, break and save if there is an error

200

In [5]:
url_list = ['https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?ref_=bo_lnav_hm_shrt',
            'https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset=200',
            'https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset=400',
            'https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset=600',
            'https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?offset=800',
            'https://www.boxofficemojo.com/chart/top_lifetime_gross/?ref_=bo_lnav_hm_shrt',
            'https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=200',
            'https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=400',
            'https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=600',
            'https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=800',
            ]

list_title_links = []

for url in url_list:
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    table = soup.find('table')
    rows = [row for row in table.find_all('tr')]  # tr tag is for rows
    
    for row in rows[1:1001]:
        link = row.find_all('td')[1].find('a')['href']
        if link not in list_title_links:
            list_title_links.append(link)
#     movies = {}
    
#     for row in rows:
#         items = row.find_all('td')
#         link = items[0].find('a')
#         title, url = link.text, link['href']
#         movies[title] = [url] + [i.text for i in items]
    
#     movies

list_title_links 

['/title/tt0499549/?ref_=bo_cso_table_1',
 '/title/tt4154796/?ref_=bo_cso_table_2',
 '/title/tt0120338/?ref_=bo_cso_table_3',
 '/title/tt2488496/?ref_=bo_cso_table_4',
 '/title/tt4154756/?ref_=bo_cso_table_5',
 '/title/tt0369610/?ref_=bo_cso_table_6',
 '/title/tt6105098/?ref_=bo_cso_table_7',
 '/title/tt0848228/?ref_=bo_cso_table_8',
 '/title/tt2820852/?ref_=bo_cso_table_9',
 '/title/tt4520988/?ref_=bo_cso_table_10',
 '/title/tt2395427/?ref_=bo_cso_table_11',
 '/title/tt1825683/?ref_=bo_cso_table_12',
 '/title/tt1201607/?ref_=bo_cso_table_13',
 '/title/tt2527336/?ref_=bo_cso_table_14',
 '/title/tt4881806/?ref_=bo_cso_table_15',
 '/title/tt2294629/?ref_=bo_cso_table_16',
 '/title/tt2771200/?ref_=bo_cso_table_17',
 '/title/tt3606756/?ref_=bo_cso_table_18',
 '/title/tt4630562/?ref_=bo_cso_table_19',
 '/title/tt1300854/?ref_=bo_cso_table_20',
 '/title/tt2293640/?ref_=bo_cso_table_21',
 '/title/tt3498820/?ref_=bo_cso_table_22',
 '/title/tt1477834/?ref_=bo_cso_table_23',
 '/title/tt0167260/?

In [6]:
len(list_title_links)

1992

In [9]:
def get_movie_dict(link):
    '''
    From BoxOfficeMojo link stub, request movie html, parse with BeautifulSoup, and
    collect 
        - title
        - dom_gross
        - intn_gross
        - ww_gross
        - minutes
        - budget
        - genre 
        - earl_rel_date
        - franchise
        - brand
    Return information as a dictionary.
    '''
    
    base_url = 'https://www.boxofficemojo.com'
    
    #Create full url to scrape
    url = base_url + link
    
    #Request HTML and parse
    response = requests.get(url)
    
    #Confirm it is working
    if response.status_code == 200:
        page = response.text
        soup = BeautifulSoup(page,"lxml")
    else:
        print('status code not equal to 200')
        
    
    try:
        title_string = soup.find('title').text
        title = title_string.split('-')[0].strip()
    except:
        title = None

    try:
        dom_gross = int(soup.find(class_='mojo-performance-summary-table').find_all('span', class_='money')[0].text.replace('$','').replace(',',''))
    except:
        dom_gross = None
        
    try:
        intn_gross = int(soup.find(class_='mojo-performance-summary-table').find_all('span', class_='money')[1].text.replace('$','').replace(',',''))
    except:
        intn_gross = None
        
    try:
        ww_gross = int(soup.find(class_='mojo-performance-summary-table').find_all('span', class_='money')[2].text.replace('$','').replace(',',''))
    except:
        ww_gross = None
        
    try:
        runtime_string = soup.find(text='Running Time')   
        runtime_string.findNext()
        runtime = runtime_string.findNext().text
        runtime = runtime.split()
        minutes = int(runtime[0])*60 + int(runtime[2])
    except:
        minutes = None
        
    try:
        budget = int(soup.find('span', text = 'Budget').find_next('span', class_='money').text.replace('$','').replace(',',''))
    except:
        budget = None
        
    try:
        genre = soup.find('span', text='Genres').find_next().text
        genre_list = genre.split()
        genre = genre_list
#         genre = ' '.join([str(elem) for elem in genre_string])
    except:
        genre = None
        
    try:
        earl_rel_date = soup.find('span', text='Earliest Release Date').find_next().text
        earl_rel_date_string = earl_rel_date.split()
        earl_rel_date = ' '.join([str(elem) for elem in earl_rel_date_string[0:3]])
    except:
        earl_rel_date = None
        
    try:
        franchise = soup.find(id='a-popover-franchisePopover').text.strip().split('\n')[0]
    except:
        franchise = 'No_Franchise'
        
    try:
        brand = soup.find(id='a-popover-brandPopover').text.strip().split('\n')[0]
    except:
        brand = 'No_Brand'

    
    #Create movie dictionary and return
    
    headers = ['movie_title', 'domestic_total_gross', 'international_total_gross','worldwide_total_gross',
           'runtime_(mins)', 'budget', 'genre','earliest_release_date', 'franchise', 'brand']
    
    movie_dict = dict(zip(headers, [title,
                                dom_gross,
                                intn_gross,
                                ww_gross,
                                minutes,
                                budget,
                                genre,
                                earl_rel_date, 
                                franchise, 
                                brand]))
#?   movie_data.append(movie_dict)
    return movie_dict

In [13]:
# ww_movies_page_info_list = []   # 1801 - 1992+1
ww_movies_page_info_list_2 = [] # 201-399
# ww_movies_page_info_list_3 = [] # 400-600+1
# ww_movies_page_info_list_4 = [] # 601-800+1
# ww_movies_page_info_list_5 = [] # 801-1000+1
# ww_movies_page_info_list_6 = [] # 1001-1200+1
# ww_movies_page_info_list_7 = [] # 1201-1400+1
# ww_movies_page_info_list_8 = [] # 1401- 1600+1
# ww_movies_page_info_list_9 = [] # 1601 - 1800+1
# ww_movies_page_info_list_10 = [] # 0 - 199 missing movie row 200 forgot stop is not inclusive

for link in list_title_links[201:400]:
    ww_movies_page_info_list_2.append(get_movie_dict(link))

In [12]:
ww_movies_page_info_list

[{'movie_title': 'Maid in Manhattan',
  'domestic_total_gross': 94011225,
  'international_total_gross': 60895468,
  'worldwide_total_gross': 154906693,
  'runtime_(mins)': 105,
  'budget': 55000000,
  'genre': ['Comedy', 'Drama', 'Romance'],
  'earliest_release_date': 'December 13, 2002',
  'franchise': 'No_Franchise',
  'brand': 'No_Brand'},
 {'movie_title': "He's Just Not That Into You",
  'domestic_total_gross': 93953653,
  'international_total_gross': 84912505,
  'worldwide_total_gross': 178866158,
  'runtime_(mins)': 129,
  'budget': None,
  'genre': ['Comedy', 'Drama', 'Romance'],
  'earliest_release_date': 'February 5, 2009',
  'franchise': 'No_Franchise',
  'brand': 'No_Brand'},
 {'movie_title': 'Master and Commander: The Far Side of the World',
  'domestic_total_gross': 93927920,
  'international_total_gross': 117694615,
  'worldwide_total_gross': 211622535,
  'runtime_(mins)': 138,
  'budget': 150000000,
  'genre': ['Action', 'Adventure', 'Drama', 'War'],
  'earliest_release

In [15]:
len(ww_movies_page_info_list)

191

In [14]:
ww_movies_page_info_list_2

[{'movie_title': 'The Meg',
  'domestic_total_gross': 145443742,
  'international_total_gross': 384815731,
  'worldwide_total_gross': 530259473,
  'runtime_(mins)': 113,
  'budget': 130000000,
  'genre': ['Action', 'Horror', 'Sci-Fi', 'Thriller'],
  'earliest_release_date': 'August 8, 2018',
  'franchise': 'No_Franchise',
  'brand': 'No_Brand'},
 {'movie_title': 'Ralph Breaks the Internet',
  'domestic_total_gross': 201091711,
  'international_total_gross': 328232251,
  'worldwide_total_gross': 529323962,
  'runtime_(mins)': 112,
  'budget': 175000000,
  'genre': ['Adventure', 'Animation', 'Comedy', 'Family', 'Fantasy', 'Sci-Fi'],
  'earliest_release_date': 'November 21, 2018',
  'franchise': 'Wreck-It Ralph',
  'brand': 'Walt Disney Animation Studios'},
 {'movie_title': 'Hotel Transylvania 3: Summer Vacation',
  'domestic_total_gross': 167510016,
  'international_total_gross': 361073758,
  'worldwide_total_gross': 528583774,
  'runtime_(mins)': 97,
  'budget': 80000000,
  'genre': ['A

In [16]:
len(ww_movies_page_info_list_2)

199

In [18]:
import pandas as pd

In [19]:
ww_movies_page_info = pd.DataFrame(ww_movies_page_info_list + ww_movies_page_info_list_2) #+ 
#                                    ww_movies_page_info_list_3 + ww_movies_page_info_list_4 + 
#                                    ww_movies_page_info_list_5 + ww_movies_page_info_list_6 + 
#                                    ww_movies_page_info_list_7 + ww_movies_page_info_list_8 +
#                                    ww_movies_page_info_list_9 + ww_movies_page_info_list_10)  #convert list of dict to df
ww_movies_page_info.set_index('movie_title', inplace=True)

ww_movies_page_info

,domestic_total_gross,international_total_gross,worldwide_total_gross,runtime_(mins),budget,genre,earliest_release_date,franchise,brand
movie_title,,,,,,,,,
Maid in Manhattan,94011225,60895468,154906693.0,105.0,55000000.0,"[Comedy, Drama, Romance]","December 13, 2002",No_Franchise,No_Brand
He's Just Not That Into You,93953653,84912505,178866158.0,129.0,NaN,"[Comedy, Drama, Romance]","February 5, 2009",No_Franchise,No_Brand
Master and Commander: The Far Side of the World,93927920,117694615,211622535.0,138.0,150000000.0,"[Action, Adventure, Drama, War]","November 14, 2003",No_Franchise,No_Brand
Flight,93772375,68000000,161772375.0,138.0,31000000.0,"[Drama, Thriller]","November 2, 2012",No_Franchise,No_Brand
The Fighter,93617009,35573860,129190869.0,116.0,25000000.0,"[Biography, Drama, Sport]","November 20, 2009",No_Franchise,No_Brand
...,...,...,...,...,...,...,...,...,...
"I, Robot",144801023,208332875,353133898.0,115.0,120000000.0,"[Action, Mystery, Sci-Fi, Thriller]","July 15, 2004",No_Franchise,No_Brand
Basic Instinct,117727224,235200000,352927224.0,127.0,49000000.0,"[Drama, Mystery, Thriller]","March 20, 1992",No_Franchise,No_Brand
Murder on the Orient Express,102826543,249967538,352794081.0,114.0,55000000.0,"[Crime, Drama, Mystery]","November 3, 2017",Agatha Christie,No_Brand
